# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [ ]:
# Setup - Run this cell first
import polars as pl
from google.colab import files
uploaded = files.upload()
uploaded = files.upload()
uploaded = files.upload()
uploaded = files.upload()
uploaded = files.upload()

Saving nyc_airlines.csv to nyc_airlines.csv


Saving nyc_airports.csv to nyc_airports.csv


Saving nyc_flights.csv to nyc_flights.csv


Saving nyc_planes.csv to nyc_planes.csv


Saving nyc_weather.csv to nyc_weather (2).csv


In [ ]:
# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values='NA')
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values='NA')
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values='NA', schema_overrides={'precip': pl.Float64, 'visib': pl.Float64})

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-1184693048.py:12: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [ ]:
# Write your SQL query here
result = ctx.execute("""
    SELECT DISTINCT carrier
    FROM airlines
""")

print(result)

shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

In [ ]:
# Write your SQL query here
result = ctx.execute("""
    SELECT dest, COUNT(*) AS num_flights
    FROM flights
    GROUP BY dest
    ORDER BY num_flights DESC
    LIMIT 10
""")

print(result)

shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [ ]:
# Write your SQL query here
result = ctx.execute("""
    SELECT *
    FROM flights
    WHERE dep_delay > 120
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [ ]:
# Write your SQL query here
result = ctx.execute("""
    SELECT origin,
           AVG(dep_delay) AS avg_dep_delay
    FROM flights
    GROUP BY origin
    ORDER BY avg_dep_delay DESC
""")

print(result)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ EWR    ┆ 15.107954     │
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
└────────┴───────────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [ ]:
# First, let's check what columns are available
result = ctx.execute("""
    SELECT *
    FROM flights
    LIMIT 5
""")
print(result)

# Now write your query to find busiest month
result = ctx.execute("""
    SELECT month, COUNT(*) AS flight_count
    FROM flights
    GROUP BY month
    ORDER BY flight_count DESC
    LIMIT 1
""")

print(result)

shape: (5, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 517      ┆ … ┆ 1400     ┆ 5    ┆ 15     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 533      ┆ … ┆ 1416     ┆ 5    ┆ 29     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 542      ┆ … ┆ 1089     ┆ 5    ┆ 40     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 544      ┆ … ┆ 1576     ┆ 5    ┆ 45     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 554      ┆ … ┆ 762      ┆ 6    ┆ 0      ┆ 2013-01-01 11:00:00 UTC │
└──────┴───────┴─────┴──────────┴───┴──────────┴──────┴────────┴───────────

### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [ ]:
# Write your SQL query here
result = ctx.execute("""
    SELECT
        carrier,
        COUNT(*) AS total_flights,
        SUM(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) AS on_time_flights,
        ROUND(
            SUM(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            2
        ) AS on_time_rate
    FROM flights
    GROUP BY carrier
    ORDER BY on_time_rate DESC
""")

print(result)


shape: (16, 4)
┌─────────┬───────────────┬─────────────────┬──────────────┐
│ carrier ┆ total_flights ┆ on_time_flights ┆ on_time_rate │
│ ---     ┆ ---           ┆ ---             ┆ ---          │
│ str     ┆ u32           ┆ i32             ┆ f64          │
╞═════════╪═══════════════╪═════════════════╪══════════════╡
│ HA      ┆ 342           ┆ 318             ┆ 92.98        │
│ AS      ┆ 714           ┆ 618             ┆ 86.55        │
│ US      ┆ 20536         ┆ 17453           ┆ 84.99        │
│ DL      ┆ 48110         ┆ 39967           ┆ 83.07        │
│ AA      ┆ 32729         ┆ 26981           ┆ 82.44        │
│ …       ┆ …             ┆ …               ┆ …            │
│ FL      ┆ 3260          ┆ 2337            ┆ 71.69        │
│ F9      ┆ 685           ┆ 490             ┆ 71.53        │
│ 9E      ┆ 18460         ┆ 12945           ┆ 70.12        │
│ EV      ┆ 54173         ┆ 35712           ┆ 65.92        │
│ YV      ┆ 601           ┆ 389             ┆ 64.73        │
└────────

## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [ ]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    f.carrier,
    a.name AS airline_name,
    f.flight,
    f.origin,
    f.dest
FROM flights f
JOIN airlines a
    ON f.carrier = a.carrier
LIMIT 20
""")

print(result)


shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [ ]:
# Write your SQL query here
result = ctx.execute("""
    SELECT f.carrier,
           AVG(2013 - p.year) AS avg_plane_age
    FROM flights f
    JOIN planes p
      ON f.tailnum = p.tailnum
    WHERE p.year IS NOT NULL
    GROUP BY f.carrier
""")

print(result)


shape: (16, 2)
┌─────────┬───────────────┐
│ carrier ┆ avg_plane_age │
│ ---     ┆ ---           │
│ str     ┆ f64           │
╞═════════╪═══════════════╡
│ WN      ┆ 9.146052      │
│ HA      ┆ 1.548387      │
│ DL      ┆ 16.372169     │
│ 9E      ┆ 7.101053      │
│ EV      ┆ 11.308998     │
│ …       ┆ …             │
│ MQ      ┆ 35.319        │
│ VX      ┆ 4.473643      │
│ F9      ┆ 4.87874       │
│ YV      ┆ 9.313758      │
│ UA      ┆ 13.207691     │
└─────────┴───────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [ ]:
# First, explore the weather table structure
result = ctx.execute("""
    SELECT *
    FROM weather
    LIMIT 5
""")
# print(result)

# Now write your join query
result = ctx.execute("""
    SELECT f.year, f.month, f.day, f.flight, f.origin, f.dest,
           f.dep_delay, w.wind_speed, w.precip
    FROM flights f
    JOIN weather w
      ON f.year = w.year
     AND f.month = w.month
     AND f.day = w.day
     AND f.hour = w.hour
     AND f.origin = w.origin
    WHERE f.dep_delay > 30
      AND (w.wind_speed > 20 OR w.precip > 0.1)
""")

print(result)


shape: (4_938, 9)
┌──────┬───────┬─────┬────────┬───┬──────┬───────────┬────────────┬────────┐
│ year ┆ month ┆ day ┆ flight ┆ … ┆ dest ┆ dep_delay ┆ wind_speed ┆ precip │
│ ---  ┆ ---   ┆ --- ┆ ---    ┆   ┆ ---  ┆ ---       ┆ ---        ┆ ---    │
│ i64  ┆ i64   ┆ i64 ┆ i64    ┆   ┆ str  ┆ i64       ┆ f64        ┆ f64    │
╞══════╪═══════╪═════╪════════╪═══╪══════╪═══════════╪════════════╪════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 21     ┆ … ┆ TPA  ┆ 47        ┆ 21.86482   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 1   ┆ 199    ┆ … ┆ LAS  ┆ 116       ┆ 21.86482   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 4090   ┆ … ┆ JAX  ┆ 39        ┆ 20.71404   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 4231   ┆ … ┆ IAD  ┆ 40        ┆ 24.16638   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 1010   ┆ … ┆ BOS  ┆ 33        ┆ 20.71404   ┆ 0.0    │
│ …    ┆ …     ┆ …   ┆ …      ┆ … ┆ …    ┆ …         ┆ …          ┆ …      │
│ 2013 ┆ 9     ┆ 22  ┆ 2347   ┆ … ┆ ATL  ┆ 41        ┆ 20.71404   ┆ 0.0    │
│ 2013 ┆ 9     ┆ 22  ┆ 5503   ┆ … ┆ PIT  ┆ 97        ┆ 20.

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [ ]:
# Write your SQL query here
result = ctx.execute("""
    SELECT p.manufacturer,
           p.model,
           COUNT(f.tailnum) AS flight_count
    FROM flights f
    JOIN planes p
      ON f.tailnum = p.tailnum
    GROUP BY p.manufacturer, p.model
    ORDER BY flight_count DESC
    LIMIT 10
""")

print(result)


shape: (10, 3)
┌───────────────────────────────┬─────────────────┬──────────────┐
│ manufacturer                  ┆ model           ┆ flight_count │
│ ---                           ┆ ---             ┆ ---          │
│ str                           ┆ str             ┆ u32          │
╞═══════════════════════════════╪═════════════════╪══════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278        │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027        │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716        │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553        │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051        │
│ BOEING                        ┆ 737-824         ┆ 13809        │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807        │
│ BOEING                        ┆ 737-7H4         ┆ 10389        │
│ BOEING                        ┆ 757-222         ┆ 9150         │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88        

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [ ]:
# Write your SQL query here
result = ctx.execute("""
WITH FlightStats AS (
    SELECT
        origin,
        dest,
        COUNT(*) AS total_flights,
        AVG(dep_delay) AS avg_dep_delay,
        SUM(CASE WHEN dep_delay > 30 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS delayed_percentage
    FROM flights
    GROUP BY origin, dest
)
SELECT
    fs.origin,
    ao.name AS origin_name,
    fs.dest,
    ad.name AS dest_name,
    fs.total_flights,
    fs.avg_dep_delay,
    fs.delayed_percentage
FROM FlightStats fs
JOIN airports ao ON fs.origin = ao.faa
JOIN airports ad ON fs.dest = ad.faa
ORDER BY fs.total_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ avg_dep_delay ┆ delayed_percen │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ ---           ┆ tage           │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ f64           ┆ ---            │
│        ┆                ┆      ┆                ┆               ┆               ┆ f64            │
╞════════╪════════════════╪══════╪════════════════╪═══════════════╪═══════════════╪════════════════╡
│ JFK    ┆ John F Kennedy ┆ LAX  ┆ Los Angeles    ┆ 11262         ┆ 8.522508      ┆ 9.829515       │
│        ┆ Intl           ┆      ┆ Intl           ┆               ┆               ┆                │
│ LGA    ┆ La Guardia     ┆ ATL  ┆ Hartsfield     ┆ 10263         ┆ 11.448621     ┆ 12.247881      │
│        ┆                ┆      ┆ Jackson        ┆               ┆         

In [ ]:
# Write your SQL query here
result = ctx.execute("""
WITH FlightStats AS (
    SELECT
        origin,
        dest,
        COUNT(*) AS total_flights,
        SUM(CASE WHEN dep_delay > 30 THEN 1 ELSE 0 END) AS delayed_over_30,
        100.0 * SUM(CASE WHEN dep_delay > 30 THEN 1 ELSE 0 END) / COUNT(*) AS pct_delayed_over_30
    FROM flights
    GROUP BY origin, dest
)
SELECT
    fs.origin,
    ao.name AS origin_name,
    fs.dest,
    ad.name AS dest_name,
    fs.total_flights,
    fs.delayed_over_30,
    fs.pct_delayed_over_30
FROM FlightStats fs
JOIN airports ao ON fs.origin = ao.faa
JOIN airports ad ON fs.dest = ad.faa
ORDER BY fs.total_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬────────────────┬───────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ delayed_over_3 ┆ pct_delayed_o │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ 0              ┆ ver_30        │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ ---            ┆ ---           │
│        ┆                ┆      ┆                ┆               ┆ i32            ┆ f64           │
╞════════╪════════════════╪══════╪════════════════╪═══════════════╪════════════════╪═══════════════╡
│ JFK    ┆ John F Kennedy ┆ LAX  ┆ Los Angeles    ┆ 11262         ┆ 1107           ┆ 9.829515      │
│        ┆ Intl           ┆      ┆ Intl           ┆               ┆                ┆               │
│ LGA    ┆ La Guardia     ┆ ATL  ┆ Hartsfield     ┆ 10263         ┆ 1257           ┆ 12.247881     │
│        ┆                ┆      ┆ Jackson        ┆               ┆         

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [ ]:
# --- Polars version: Top 10 routes with names & performance metrics ---

# Join flights -> airports twice to bring in origin & destination names
routes_with_names = (
    flights
    .join(
        airports.select(['faa', 'name']).rename({'faa': 'origin', 'name': 'origin_name'}),
        on='origin',
        how='left'
    )
    .join(
        airports.select(['faa', 'name']).rename({'faa': 'dest', 'name': 'dest_name'}),
        on='dest',
        how='left'
    )
)

# Aggregate by route, compute metrics
routes_perf = (
    routes_with_names
    .group_by(['origin', 'origin_name', 'dest', 'dest_name'])
    .agg([
        pl.count().alias('total_flights'),
        pl.col('dep_delay').mean().alias('avg_dep_delay'),
        # count of flights with dep_delay > 30 (ignore nulls)
        (pl.col('dep_delay').gt(30).cast(pl.UInt32)).sum().alias('delayed_over_30')
    ])
    .with_columns(
        (pl.col('delayed_over_30') * 100.0 / pl.col('total_flights'))
        .alias('pct_delayed_over_30')
    )
    .sort('total_flights', descending=True)
)

# Show the Top 10 routes by total flights (with average dep delay)
top10_routes_avg = (
    routes_perf
    .select(['origin', 'origin_name', 'dest', 'dest_name', 'total_flights', 'avg_dep_delay'])
    .head(10)
)

# Show the Top 10 routes by total flights (with % delayed > 30)
top10_routes_pct = (
    routes_perf
    .select(['origin', 'origin_name', 'dest', 'dest_name', 'total_flights', 'pct_delayed_over_30'])
    .head(10)
)

print("Top 10 routes (avg dep delay):")
print(top10_routes_avg)

print("\nTop 10 routes (% delayed > 30 min):")
print(top10_routes_pct)


Top 10 routes (avg dep delay):
shape: (10, 6)
┌────────┬─────────────────────┬──────┬────────────────────────────┬───────────────┬───────────────┐
│ origin ┆ origin_name         ┆ dest ┆ dest_name                  ┆ total_flights ┆ avg_dep_delay │
│ ---    ┆ ---                 ┆ ---  ┆ ---                        ┆ ---           ┆ ---           │
│ str    ┆ str                 ┆ str  ┆ str                        ┆ u32           ┆ f64           │
╞════════╪═════════════════════╪══════╪════════════════════════════╪═══════════════╪═══════════════╡
│ JFK    ┆ John F Kennedy Intl ┆ LAX  ┆ Los Angeles Intl           ┆ 11262         ┆ 8.522508      │
│ LGA    ┆ La Guardia          ┆ ATL  ┆ Hartsfield Jackson Atlanta ┆ 10263         ┆ 11.448621     │
│        ┆                     ┆      ┆ Int…                       ┆               ┆               │
│ LGA    ┆ La Guardia          ┆ ORD  ┆ Chicago Ohare Intl         ┆ 8857          ┆ 10.740758     │
│ JFK    ┆ John F Kennedy Intl ┆ SFO  ┆ San F

/tmp/ipython-input-1024550309.py:23: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias('total_flights'),
